In [ ]:
# mount google colab
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# install packages
!pip install transformers datasets evaluate
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.3 MB/s eta 0:00:00


In [ ]:
# login to huggingface
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# import dataset wich should be used for finetuning
from datasets import load_dataset

right_dataset = load_dataset('csv', data_files='add_path_to_dataset/allsides_data_right_heading.csv')
right_dataset = right_dataset['train']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# split data into trainings and test split
right_dataset = right_dataset.train_test_split(test_size=0.2)

# Preprocess

In [ ]:
# import tokenizer for the pretrained model wich should be used
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

In [ ]:
# function to apply tokenizing
def preprocess_function(examples):
    return tokenizer(examples["heading"]) #define column wich should be used for finetuning

In [ ]:
# tokenize dataset
tokenized_right_dataset = right_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=right_dataset["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/6464 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1617 [00:00<?, ? examples/s]

In [ ]:
# chunk size for concatenation
block_size = 128

# function to concatenate data
def group_texts(examples):
    # concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # split by chunks of block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
# apply concatenation to dataset
lm_dataset = tokenized_right_dataset.map(group_texts, batched=True, num_proc=4)
#lm_dataset['train'][0]

In [ ]:
# import the Trainer and Datacollector for finetuning purposes
from transformers import DataCollatorForLanguageModeling

# add pad token to tokenizer
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) #load datacollector

# Train

In [ ]:
# import the pretrained model wich should be finetuned
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("distilgpt2")

In [ ]:
#finetune the pretrained model on the dataset

training_args = TrainingArguments(
    output_dir="distillgpt2_right_heading_torch",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    num_train_epochs=8
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

Cloning https://huggingface.co/tobijen/distillgpt2_right_heading_torch into local empty directory.


Download file pytorch_model.bin:   0%|          | 8.16k/312M [00:00<?, ?B/s]

Download file runs/Jul25_10-50-26_d065ce15b042/events.out.tfevents.1690282235.d065ce15b042.6170.0: 100%|######…

Download file runs/Jul25_11-43-59_d065ce15b042/events.out.tfevents.1690285442.d065ce15b042.6170.8: 100%|######…

Clean file runs/Jul25_10-50-26_d065ce15b042/events.out.tfevents.1690282235.d065ce15b042.6170.0:  15%|#5       …

...

Clean file runs/Jul25_10-50-26_d065ce15b042/events.out.tfevents.1690282472.d065ce15b042.6170.1: 100%|#########…

Clean file pytorch_model.bin:   0%|          | 1.00k/312M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,4.891377
2,No log,4.778655
3,No log,4.726338
4,No log,4.705027
5,No log,4.692724
6,4.799000,4.686180
7,4.799000,4.684093
8,4.799000,4.682597


TrainOutput(global_step=768, training_loss=4.693555990854899, metrics={'train_runtime': 77.7364, 'train_samples_per_second': 78.625, 'train_steps_per_second': 9.88, 'total_flos': 199630717452288.0, 'train_loss': 4.693555990854899, 'epoch': 8.0})

In [ ]:
# get the perplexity for the model
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 108.05


In [ ]:
# push finetuned model to huggingface repo
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/312M [00:00<?, ?B/s]

Upload file runs/Aug01_11-01-01_5d751d389ba8/events.out.tfevents.1690887758.5d751d389ba8.1357.0:   0%|        …

Upload file runs/Aug01_11-01-01_5d751d389ba8/events.out.tfevents.1690887859.5d751d389ba8.1357.1:   0%|        …

To https://huggingface.co/tobijen/distillgpt2_right_heading_torch
   a1e372a..b2f6ee0  main -> main

   a1e372a..b2f6ee0  main -> main

To https://huggingface.co/tobijen/distillgpt2_right_heading_torch
   b2f6ee0..8e0c967  main -> main

   b2f6ee0..8e0c967  main -> main



'https://huggingface.co/tobijen/distillgpt2_right_heading_torch/commit/b2f6ee033b91bb6c97cb2e2537ea102c27da4751'

In [ ]:
# save model locally
trainer.save_model("path_to_where_the_model_should_be_saved_locally/distillgpt2_right_heading_torch")

In [ ]:
# Inputs to generate text for
prompts = {
    "social_1": "Gay marriage is",
    "social_2": "Abortion is",
    "education_1": "Public education is",
    "education_2": "Charter schools are",
    "economic_1":"The increase of taxes is",
    "economic_2":"Government regulations on businesses are",
}

In [ ]:
# generate text with the pipeline module from the transformers library
from transformers import pipeline

generated_text_dict = {}
for key, prompt in prompts.items():
  print(key, " => ", prompt)
  generator = pipeline("text-generation", model="tobijen/distillgpt2_right_heading_torch", tokenizer=tokenizer)
  generated_text = generator(prompt)
  generated_text_dict[key] = generated_text[0]["generated_text"]
  print(generated_text)

social_1  =>  Gay marriage is


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (50) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': "Gay marriage is ‘conscience’ — that’s right, says Michael Cohen, 'I will fight it for what’s right’Racial segregation is a great way to get involved in politicsThe Post and MailOnline"}]
social_2  =>  Abortion is


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Abortion is a woman’s right. Don’t Miss America 1.0A new report finds Romney would win big among 2020 primary voters, likely ahead of November's debateNo, Mitt’s Speech Doesn’t Really"}]
education_1  =>  Public education is


...

[{'generated_text': "Charter schools are 'doing nothing' on illegal immigrantsUS to have more transgender students enroll in schools in 2024, ending long school-mandated gender change mandateSupreme Court's January decision in North Carolina's 2016 ruling banning transgender athletes from attending states"}]
economic_1  =>  The increase of taxes is


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "The increase of taxes is on the order of 'derelict'Clinton aides say former ambassador says she could be arrested by Trump: ‘Can't I have this debate?'Clinton to back off calls for impeachment, but Obama administration has yet to"}]
economic_2  =>  Government regulations on businesses are


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Government regulations on businesses are more complex Than just ObamacareTrump’s budget proposal, first seen as bad for the American publicTrump to take a different look at tax reformAjit Pai: The ‘I’m Just Don’t'}]


In [ ]:
generated_text_dict

{'social_1': "Gay marriage is ‘conscience’ — that’s right, says Michael Cohen, 'I will fight it for what’s right’Racial segregation is a great way to get involved in politicsThe Post and MailOnline",
 'social_2': "Abortion is a woman’s right. Don’t Miss America 1.0A new report finds Romney would win big among 2020 primary voters, likely ahead of November's debateNo, Mitt’s Speech Doesn’t Really",
 'education_1': "Public education is already being challenged in high school by schoolteachers' group, Trump defends Romney's health, claims'very good' for the economy, warns 'nothing will happen'A new poll released shows Hillary Clinton trailing Iowa Republican, but not",
 'education_2': "Charter schools are 'doing nothing' on illegal immigrantsUS to have more transgender students enroll in schools in 2024, ending long school-mandated gender change mandateSupreme Court's January decision in North Carolina's 2016 ruling banning transgender athletes from attending states",
 'economic_1': "The i

In [ ]:
# add the generated text to a json file, wich is used for the evaluation
import json
import os

def write_dict_to_json_file(file_path, data):
    # Check if the JSON file exists
    if os.path.exists(file_path):
        # If the file exists, load the existing data
        with open(file_path, 'r', encoding='utf-8') as file:
            existing_data = json.load(file)
    else:
        # If the file does not exist, create an empty dictionary
        existing_data = {}

    # Update the existing dictionary with the new data
    existing_data.update(data)

    # Write the updated dictionary back to the JSON file
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(existing_data, file, ensure_ascii=False)

data_to_append = {
    "distillgpt2_right_headlines": generated_text_dict
}

# write data to json file
write_dict_to_json_file('path_to_json_file/generated_texts.json', data_to_append)